This script will try to reproduce the sequence label-scrambling observed in [this](https://github.com/evogytis/EBOV-rates-project-2016/blob/master/XMLs/GireWA_hkySCCS_20M_1.xml) XML.

First, we will take the [strain mapping file](https://github.com/evogytis/EBOV-rates-project-2016/blob/master/data/strain_map.csv) (see [this notebook](https://github.com/evogytis/EBOV-rates-project-2016/blob/master/notebooks/EBOV_rates_project_2016.ipynb) for details), which tells us which sequences in GenBank match sequences in the original XML, and produce an [unique matching](https://github.com/evogytis/EBOV-rates-project-2016/blob/master/data/unique_strain_map.csv) by resolving the ambiguities. The reason such ambiguities exist is that many sequences are identical, hence one cannot exactly which labels got switched.

In [38]:
Map <- read.table("../data/strain_map.csv", header = TRUE, sep = ",")
create.unique.mapping <- function(non.unique.mapping){
  ## 'non.unique.mapping' is a data.frame, where the second column may have multiple entries (matches)
  ## The goal here is to produce (one of the possbible) unique mappings
  ## For this we will iterate over the rows of the table:
  ## if each column has only one entry, then this is an unique row; add it to the output and
  ## exclude the ID in the second column from the remaining rows and proceed
  ## The algorithm terminates when it reaches the last row and there's only one possible matching
  ## The output is a 2 X N table with only unique mappings
  
  # 0) First, let's get the unique matchings out of the way:
  nmatches <- function(x) unlist(lapply(strsplit(as.character(x), ";"), length))
  Nmatch <- nmatches(non.unique.mapping[ , 2])
  res0 <- non.unique.mapping[which(Nmatch < 2), ] ## unique matches, 1st part of the answer
      
  # 1) Now let's iterate over the entries with more than one match and always take the FIRST one;    
  Toresolve <- non.unique.mapping[which(Nmatch > 1), ] ## These are the ones that need resolving
  K <- nrow(Toresolve)
  res1 <- data.frame(matrix(NA, ncol = 2, nrow = K))
  names(res1) <- names(res0) <- c("correct", "matching")
  res1[, 1] <- Toresolve[, 1]
  Toresolve.list <- strsplit(as.character(Toresolve[, 2]), ";")
  for(row in 1:K){
    entry <- Toresolve.list[[row]][1]
    res1[row, 2] <- entry
    Toresolve.list <- lapply(Toresolve.list, function(x) x[which(x != entry)])
  }
  res.final <- rbind(res0, res1)
  names(res.final) <- c("correct", "matching")
  return(res.final) 
}
uniqueMap <- create.unique.mapping(Map[, c(1, 3)])

How many sequences have the correct label?

In [39]:
sum( apply(uniqueMap, 1, function(row) row[1] == row[2]) )

[1] 9

In [40]:
write.csv(uniqueMap, file = "../data/unique_strain_map.csv", row.names = FALSE)

Now let us take the [correct alignment](https://github.com/evogytis/EBOV-rates-project-2016/blob/master/alignments/Gire_Mali_old.fasta) and scramble the labels as per the unique mapping we just created.

In [43]:
library(ape)
Aln <- read.dna(file = "../alignments/Gire_Mali_old.fasta", format = "fasta") ## correct alignment
head(rownames(Aln), 10)

[1] "KP260799|Makona-Mali-DPR1|2014-10-23"
 [2] "KP260800|Makona-Mali-DPR2|2014-11-12"
 [3] "KP260801|Makona-Mali-DPR3|2014-11-21"
 [4] "KP260802|Makona-Mali-DPR4|2014-11-12"
 [5] "KM233035|Makona-EM104|2014-06-02"    
 [6] "KM233073|Makona-G3786|2014-06-14"    
 [7] "KM233078|Makona-G3796|2014-06-15"    
 [8] "KM233042|Makona-EM119|2014-06-03"    
 [9] "KM233040|Makona-EM113|2014-06-03"    
[10] "KM233049|Makona-G3707|2014-06-06"

In [ ]:
N <- nrow(Aln)
newNames <- rep(NA, N)
for(i in 1:N){
  pos1 <- grep(uniqueMap$correct[i], rownames(Aln))
  pos2 <- grep(uniqueMap$matching[i], rownames(Aln))
  newNames[pos1] <- rownames(Aln)[pos2]
}
aln.scrambled <- Aln 
rownames(aln.scrambled) <- newNames
write.dna(aln.scrambled, file = "../alignments/Gire_Mali_old_shuffled.fasta", format = "fasta")